In [61]:
from torchvision.transforms import v2
from torchvision import tv_tensors
from PIL import Image
import numpy as np
import torch

from torchvision.utils import draw_bounding_boxes
from torchvision.ops import box_convert

from matplotlib import pyplot as plt

In [62]:
image_path = '../data/images/IMG_20241207_163835.jpg'
label_path = image_path.replace('images', 'labels').replace('.jpg', '.txt')
image = Image.open(image_path)

yolo_boxes = []
yolo_classes = []
with open(label_path, 'r') as f:
    line = f.readline()
    while line:
        class_id, x_center, y_center, width, height = map(float, line.strip().split())
        yolo_boxes.append([x_center, y_center, width, height])
        yolo_classes.append([class_id])
        line = f.readline()

W, H = image.size

In [ ]:
np.array(image).shape

In [ ]:
torch_image = torch.from_numpy(np.array(image).transpose(2, 0, 1))
torch_image.shape

In [65]:
tv_boxes = torch.from_numpy(np.array(yolo_boxes).copy())
tv_boxes[:, [0, 2]] *= W
tv_boxes[:, [1, 3]] *= H

In [66]:
tv_boxes_xyxy = box_convert(tv_boxes, in_fmt='xywh', out_fmt='xyxy')

In [ ]:
dbb = draw_bounding_boxes(torch_image, tv_boxes_xyxy, labels=None, colors='red', fill=True, width=10)
plt.imshow(dbb.numpy().transpose(1, 2, 0))

In [68]:
transform = v2.Compose([
    v2.RandomIoUCrop(min_scale=0.0001),
    v2.Resize((512, 512)),
    v2.RandomHorizontalFlip(0.5),
    v2.RandomVerticalFlip(0.5),
    v2.ColorJitter(),
    v2.ToDtype(torch.float32, scale=True)
])

In [69]:
tvt_image = tv_tensors.Image(torch_image)
tvt_boxes = tv_tensors.BoundingBoxes(tv_boxes, format='XYWH', canvas_size=torch_image.shape[-2:], dtype=torch.float32)

In [70]:
trafo_image, trafo_boxes = transform(tvt_image, tvt_boxes)
sane_boxes = v2.SanitizeBoundingBoxes()({'labels':trafo_boxes})

In [ ]:
trafo_boxes_xyxy = box_convert(sane_boxes['labels'], in_fmt='xywh', out_fmt='xyxy')
dbb = draw_bounding_boxes(trafo_image, trafo_boxes_xyxy, labels=None, colors='red', fill=True, width=10)
plt.imshow(dbb.numpy().transpose(1, 2, 0))